In [26]:
import pandas as pd
import os
import regex as re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from gensim.models import Word2Vec
import numpy as np
import gensim.downloader as api

In [27]:
data = pd.read_csv("/Users/aditya/Documents/GitHub/tegger-challenge/data.csv").iloc[:,1:]
data.head()

,all,theme,title,contents,argentina,australia,brazil,canada,china,france,...,japan,republic of korea,mexico,russia,saudi arabia,south africa,turkey,united kingdom,united states,european union
0,musician tackle *USA red tape musician group t...,entertainment,Musicians to tackle US red tape,Musicians' groups are to tackle US visa regula...,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,1,1
1,u2 desire number one u2 three prestigious gram...,entertainment,U2's desire to be number one,"U2, who have won three prestigious Grammy Awar...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
2,rocker doherty onstage fight rock singer pete ...,entertainment,Rocker Doherty in on-stage fight,Rock singer Pete Doherty has been involved in ...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,snicket top *USA box office chart film adaptat...,entertainment,Snicket tops US box office chart,The film adaptation of Lemony Snicket novels h...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
4,ocean twelve raid box office ocean twelve crim...,entertainment,Ocean's Twelve raids box office,"Ocean's Twelve, the crime caper sequel starrin...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [28]:
# cv=CountVectorizer(min_df=2,stop_words='english', max_df=0.85, ngram_range=(1,3))
# word_count_vector=cv.fit_transform(data['all'])

In [29]:
# feature_names=cv.get_feature_names()

In [30]:
# vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,3), min_df=2, max_df=0.85)
# tfidf = vectorizer.fit_transform(data['all'])

In [31]:
corpus = [entry.split(" ") for entry in data['all']]

In [32]:
# model = Word2Vec(corpus, min_count=2,size=50000,workers=3, window =5, sg = 0)

In [33]:
# model.save("w2v_saved.model")
model = Word2Vec.load("w2v_saved.model")
# model_glove = api.load("glove-wiki-gigaword-100")
# context = {}
# context.update(model_glove)
# context.update(model)

In [34]:
g20 = "Argentina, Australia, Brazil, Canada, China, France, Germany, India, Indonesia, Italy, Japan, Republic of Korea, Korea, Mexico, Russia, Saudi Arabia, Arabia, South Africa, Africa, Turkey, United Kingdom, UK, United States, America, USA, European Union, EU, Europe".lower()
g20 = g20.split(", ")

In [42]:
def get_mean_vector(word2vec_model, words):
    words = [word for word in words if word in word2vec_model.wv.vocab.keys()]
    if len(words) >= 1:
        avg1= np.mean(word2vec_model[words], axis=0)
        return avg1
    else:
        return []

In [43]:
data['mean_emb'] = data['all'].apply(lambda x: get_mean_vector(model, x.split(" ")))

In [44]:
data.head()

,all,theme,title,contents,argentina,australia,brazil,canada,china,france,...,republic of korea,mexico,russia,saudi arabia,south africa,turkey,united kingdom,united states,european union,mean_emb
0,musician tackle *USA red tape musician group t...,entertainment,Musicians to tackle US red tape,Musicians' groups are to tackle US visa regula...,0,0,0,0,1,0,...,0,0,0,0,0,0,1,1,1,"[0.010030553, 0.010555822, 0.0056487704, -0.00..."
1,u2 desire number one u2 three prestigious gram...,entertainment,U2's desire to be number one,"U2, who have won three prestigious Grammy Awar...",0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,"[0.009995017, 0.012596794, 0.0047847545, -0.00..."
2,rocker doherty onstage fight rock singer pete ...,entertainment,Rocker Doherty in on-stage fight,Rock singer Pete Doherty has been involved in ...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0.0095635895, 0.012760582, 0.004244135, -0.00..."
3,snicket top *USA box office chart film adaptat...,entertainment,Snicket tops US box office chart,The film adaptation of Lemony Snicket novels h...,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,"[0.009604343, 0.014471119, 0.0036928602, -0.00..."
4,ocean twelve raid box office ocean twelve crim...,entertainment,Ocean's Twelve raids box office,"Ocean's Twelve, the crime caper sequel starrin...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,"[0.0082686655, 0.012306145, 0.003344359, -0.00..."


In [45]:
data.to_pickle('/Users/aditya/Documents/GitHub/tegger-challenge/mean_embeddings.pkl')

In [52]:
g20_mean_emb = {}
for each in g20:
    try:
        g20_mean_emb[each] = model['*'+each.upper()]
    except:
        continue


In [53]:
g20_mean_emb

{'argentina': array([ 0.01205488,  0.01540329,  0.00587998, ..., -0.03364547,
        -0.02346341,  0.01895906], dtype=float32),
 'australia': array([ 0.0120225 ,  0.01579212,  0.00597072, ..., -0.03384686,
        -0.0229663 ,  0.01896352], dtype=float32),
 'brazil': array([ 0.00753566,  0.00865833,  0.00401651, ..., -0.02044253,
        -0.01490953,  0.01099955], dtype=float32),
 'canada': array([ 0.00579164,  0.00740887,  0.00269136, ..., -0.01600016,
        -0.01082051,  0.00921129], dtype=float32),
 'china': array([ 0.01522709,  0.01920916,  0.00810206, ..., -0.04266999,
        -0.02943521,  0.02328844], dtype=float32),
 'france': array([ 0.01525989,  0.0218841 ,  0.00681947, ..., -0.04391771,
        -0.02820129,  0.02574923], dtype=float32),
 'germany': array([ 0.01111386,  0.01362935,  0.00545584, ..., -0.03046325,
        -0.0210385 ,  0.01710157], dtype=float32),
 'india': array([ 0.01481213,  0.01650877,  0.008064  , ..., -0.03975716,
        -0.02910383,  0.02112232], dty